In [9]:
import torch
import argparse
import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models, transforms

# class FeatureExtractor():
#     """ Class for extracting activations and
#     registering gradients from targetted intermediate layers """

#     def __init__(self, model, target_layers):
#         self.model = model
#         self.target_layers = target_layers
#         self.gradients = []

#     def save_gradient(self, grad):
#         self.gradients.append(grad)

#     def __call__(self, x):
#         outputs = []
#         self.gradients = []
#         for name, module in self.model._modules.items():
#             x = module(x)
#             if name in self.target_layers:
#                 x.register_hook(self.save_gradient)
#                 outputs += [x]
#         return outputs, x

class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)

        return target_activations, x

# def preprocess_image(img):
#     normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])
#     preprocessing = transforms.Compose([
#         transforms.ToTensor(),
#         normalize,
#     ])
#     return preprocessing(img.copy()).unsqueeze(0)

def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        features, output = self.extractor(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()
        
        one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features[-1]
        target = target.cpu().data.numpy()[0, :]

        weights = np.mean(grads_val, axis=(2, 3))[0, :]
        cam = np.zeros(target.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):
            cam += w * target[i, :, :]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, input_img.shape[2:])
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam


class GuidedBackpropReLU(Function):
    @staticmethod
    def forward(self, input_img):
        positive_mask = (input_img > 0).type_as(input_img)
        output = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), input_img, positive_mask)
        self.save_for_backward(input_img, output)
        return output

    @staticmethod
    def backward(self, grad_output):
        input_img, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input_img > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img),
                                   torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), grad_output,
                                                 positive_mask_1), positive_mask_2)
        return grad_input


class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        def recursive_relu_apply(module_top):
            for idx, module in module_top._modules.items():
                recursive_relu_apply(module)
                if module.__class__.__name__ == 'ReLU':
                    module_top._modules[idx] = GuidedBackpropReLU.apply

        # replace ReLU with GuidedBackpropReLU
        recursive_relu_apply(self.model)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        input_img = input_img.requires_grad_(True)

        output = self.forward(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()

        one_hot = torch.sum(one_hot * output)
        one_hot.backward(retain_graph=True)

        output = input_img.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-cuda', action='store_true', default=False,
                        help='Use NVIDIA GPU acceleration')
    parser.add_argument('--image-path', type=str, default='./examples/both.png',
                        help='Input image path')
    args = parser.parse_args()
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print("Using GPU for acceleration")
    else:
        print("Using CPU for computation")

    return args

def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)

if __name__ == '__main__':
    """ python grad_cam.py <path_to_image>
    1. Loads an image with opencv.
    2. Preprocesses it for VGG19 and converts to a pytorch variable.
    3. Makes a forward pass to find the category index with the highest score,
    and computes intermediate activations.
    Makes the visualization. """

    args = get_args()

    model = models.resnet50(pretrained=True)
    grad_cam = GradCam(model=model, feature_module=model.layer4, \
                       target_layer_names=["2"], use_cuda=args.use_cuda)

    img = cv2.imread(args.image_path, 1)
    img = np.float32(img) / 255
    # Opencv loads as BGR:
    img = img[:, :, ::-1]
    input_img = preprocess_image(img)

    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    target_category = None
    grayscale_cam = grad_cam(input_img, target_category)

    grayscale_cam = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
    cam = show_cam_on_image(img, grayscale_cam)

    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=args.use_cuda)
    gb = gb_model(input_img, target_category=target_category)
    gb = gb.transpose((1, 2, 0))

    cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
    cam_gb = deprocess_image(cam_mask*gb)
    gb = deprocess_image(gb)

    cv2.imwrite("cam.jpg", cam)
    cv2.imwrite('gb.jpg', gb)
    cv2.imwrite('cam_gb.jpg', cam_gb)

usage: ipykernel_launcher.py [-h] [--use-cuda] [--image-path IMAGE_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/sol107/.local/share/jupyter/runtime/kernel-fc3a6003-46b7-48ae-b31d-39a016d3b05b.json


SystemExit: 2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# shin's gradcam

In [6]:
import sys
import json
import os
from torchvision import models
import shutil

sys.path.insert(0, 'src')

# from etl import get_data
# from process import getRandomImg
# from viz import ImplementGrad

def main(targets):
    if ('data' in targets) or ('all' in targets) or ('test' in targets):
        with open('config/data-params.json') as fh:
            data_cfg = json.load(fh)
        get_data(**data_cfg)
        
    if ('grad' in targets) or ('all' in targets) or ('test' in targets):
        with open('config/image-params.json') as fh:
            data_img = json.load(fh)
            
        if not os.path.exists("results"):
            os.mkdir("results")
        img = getRandomImg(**data_img)
        
        model = models.resnet50(pretrained=True)
        
        
        grad = ImplementGrad(model=model, results_path = data_img["out_dir"])
        grad.heat_map(img)
        grad.gb(img)
        
    if ('clean' in targets):
        if os.path.exists("results"):
            shutil.rmtree('results/')
        if os.path.exists("data"):
            shutil.rmtree('data/')
    return

if __name__ == '__main__':
    targets = sys.argv[1:]
    main(targets)

In [8]:
main('result_path')

In [ ]:
import sys
import json

sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/analysis')
sys.path.insert(0, 'src/model')
sys.path.insert(0, 'src/util')
sys.path.insert(0, 'test/')

from src.util import download_annotations
from src.util import run_demo
from src.model import model
from src.util import coco_dataloader
from src.util import coco_dict
from test import test_model

import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision
import torch
from collections import defaultdict

from pycocotools.coco import COCO


def main(targets):
    '''
    runs basic demo for COCO api.
    
    configure filepaths based on data-params.json, however all should work by default.
    '''
    if 'data' in targets:
        with open('config/data-params.json') as fh:
                data_cfg = json.load(fh)
                print(data_cfg['dataDirJSON'])

        # load paths
        dataDirJSON = data_cfg['dataDirJSON']
        dataType = data_cfg['dataTypeVal']
        annDir = data_cfg['annDir'].format(dataDirJSON)
        annZipFile = data_cfg['annZipFile'].format(dataDirJSON, dataType)
        annFile = data_cfg['annFile'].format(annDir, dataType)
        annURL = data_cfg['annURL'].format(dataType)
        capAnnFile = data_cfg['capAnnFile'].format(dataDirJSON, dataType)

        # run coco_demo.py function run_demo
        run_demo(dataDirJSON, dataType, annDir, annZipFile, annFile, annURL, capAnnFile)
        
    elif 'test_meth' in targets:
        with open('test/test-data-params.json') as fh:
                data_cfg = json.load(fh)
                print(data_cfg['dataDirJSON'])

        # load paths
        dataDirJSON = data_cfg['dataDirJSON']
        dataType = data_cfg['dataTypeVal']
        annDir = data_cfg['annDir'].format(dataDirJSON)
        annZipFile = data_cfg['annZipFile'].format(dataDirJSON, dataType)
        annFile = data_cfg['annFile'].format(annDir, dataType)
        annURL = data_cfg['annURL'].format(dataType)
        capAnnFile = data_cfg['capAnnFile'].format(dataDirJSON, dataType)

        # run coco_demo.py function run_demo
        run_test_demo(dataDirJSON, dataType, annDir, annZipFile, annFile, annURL, capAnnFile)
        
    elif 'train' in targets:
        with open('config/data-params.json') as fh:
            data_cfg = json.load(fh)
        
        with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)

        # build paths
        TRAIN_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeTrain']
        VAL_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeVal']
        JSON_TRAIN_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeTrain'])
        JSON_VAL_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeVal'])
        
        # initiate coco for dict
        coco = COCO(JSON_TRAIN_PATH)
        
        # build encoder/decoder dicts 
        encoder_dict, decoder_dict = coco_dict.build_dicts(coco)
        
        # change to gpu 
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print("COMPUTING AS: {}".format(device))
        
        # transforms 
        transform = transforms.Compose([ 
                transforms.Resize([240,240]),
                transforms.ToTensor()
        ])
        
        # define CNN
        net = model.Net()
        net = net.to(device) # to gpu
        
        # initiate custom torchvision dset
        coco_train = coco_dataloader.CocoDetection(root = TRAIN_PATH, annFile = JSON_TRAIN_PATH, transform = transform )
        
        # dataloaders
        trainloader = torch.utils.data.DataLoader(coco_train, batch_size=5,
                                       shuffle=True, num_workers=2)

        # run the model
        model.run_model(
            loader = trainloader,
            net = net,
            epochs = model_cfg['epochs'],
            lr = model_cfg['lr'],
            momentum = model_cfg['momentum'],
            modelDir = model_cfg['modelDir'],
            device = device
        )
        
    elif 'test' in targets:
        with open('config/data-params.json') as fh:
            data_cfg = json.load(fh)
        # load the modelDir
        with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)
        
        # build paths
        TRAIN_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeTrain']
        VAL_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeVal']
        JSON_TRAIN_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeTrain'])
        JSON_VAL_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeVal'])
        
        # transforms 
        transform = transforms.Compose([ 
                transforms.Resize([240,240]),
                transforms.ToTensor()
        ])
        
        # load test data
        coco_test =  coco_dataloader.CocoDetection(root = VAL_PATH, annFile = JSON_VAL_PATH, transform = transform )
        testloader = torch.utils.data.DataLoader(coco_test, batch_size=5,
                               shuffle=True, num_workers=2)
        
        # change to gpu 
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print("COMPUTING AS: {}".format(device))
        
        net = model.Net()
        net = net.to(device)
        
        print('Loading model from {}...'.format(model_cfg['modelDir']))
        net.load_state_dict(torch.load(model_cfg['modelDir']))
        
        model.test_model(net, testloader, device)
        
        elif 'method7' in targets:
            with open('test/test-data-params.json') as fh:
                method7_config = json.load(fh)
                
            with open('config/model-params.json') as fh:
                model_cfg = json.load(fh)
                
            #load in data
            inputs, labels = test_model.load_testdata(method7_config['train_inputs'],method7_config['train_labels'])
            #transforms
            transform = transforms.Compose([transfroms.Resize([240, 240]), transforms.ToTensor()])
            
            # change to gpu
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            print('Computing as: {}'.format(device))
            
            net = model.Net()
            net = net.to(device)

            #run the model
            test_model.run_model(
                input_tensors = inputs,
                label_tensors = labels,
                net = net,
                epochs = model_cfg['epochs'],
                lr = model_cfg['lr'],
                momentum = model_cfg['momentum'],
                modelDir = m7['modelDir'],
                device = device)
if __name__ == '__main__':
    targets = sys.argv[1:]
    main(targets)

In [ ]:
import sys
import json

sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/analysis')
sys.path.insert(0, 'src/model')
# sys.path.insert(0, 'src/util')
sys.path.insert(0, 'test/')

# from src.util import download_annotations
# from src.util import run_demo
from src.model import model
# from src.util import coco_dataloader
# from src.util import coco_dict
from test import test_model

import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision
import torch
from collections import defaultdict

# from pycocotools.coco import COCO


def main(targets):
    '''
    runs basic demo for COCO api.
    
    configure filepaths based on data-params.json, however all should work by default.
    '''
    if 'data' in targets:
        with open('config/data-params.json') as fh:
                data_cfg = json.load(fh)
                print(data_cfg['dataDirJSON'])

        # load paths
        dataDirJSON = data_cfg['dataDirJSON']
        dataType = data_cfg['dataTypeVal']
        annDir = data_cfg['annDir'].format(dataDirJSON)
        annZipFile = data_cfg['annZipFile'].format(dataDirJSON, dataType)
        annFile = data_cfg['annFile'].format(annDir, dataType)
        annURL = data_cfg['annURL'].format(dataType)
        capAnnFile = data_cfg['capAnnFile'].format(dataDirJSON, dataType)

        # run coco_demo.py function run_demo
        run_demo(dataDirJSON, dataType, annDir, annZipFile, annFile, annURL, capAnnFile)
        
    elif 'test_meth' in targets:
        with open('test/test-data-params.json') as fh:
                data_cfg = json.load(fh)
                print(data_cfg['dataDirJSON'])

        # load paths
        dataDirJSON = data_cfg['dataDirJSON']
        dataType = data_cfg['dataTypeVal']
        annDir = data_cfg['annDir'].format(dataDirJSON)
        annZipFile = data_cfg['annZipFile'].format(dataDirJSON, dataType)
        annFile = data_cfg['annFile'].format(annDir, dataType)
        annURL = data_cfg['annURL'].format(dataType)
        capAnnFile = data_cfg['capAnnFile'].format(dataDirJSON, dataType)

        # run coco_demo.py function run_demo
        run_test_demo(dataDirJSON, dataType, annDir, annZipFile, annFile, annURL, capAnnFile)
        
    elif 'train' in targets:
        with open('config/data-params.json') as fh:
            data_cfg = json.load(fh)
        
        with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)

        # build paths
        TRAIN_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeTrain']
        VAL_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeVal']
        JSON_TRAIN_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeTrain'])
        JSON_VAL_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeVal'])
        
        # initiate coco for dict
        coco = COCO(JSON_TRAIN_PATH)
        
        # build encoder/decoder dicts 
        encoder_dict, decoder_dict = coco_dict.build_dicts(coco)
        
        # change to gpu 
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print("COMPUTING AS: {}".format(device))
        
        # transforms 
        transform = transforms.Compose([ 
                transforms.Resize([240,240]),
                transforms.ToTensor()
        ])
        
        # define CNN
        net = model.Net()
        net = net.to(device) # to gpu
        
        # initiate custom torchvision dset
        coco_train = coco_dataloader.CocoDetection(root = TRAIN_PATH, annFile = JSON_TRAIN_PATH, transform = transform )
        
        # dataloaders
        trainloader = torch.utils.data.DataLoader(coco_train, batch_size=5,
                                       shuffle=True, num_workers=2)

        # run the model
        model.run_model(
            loader = trainloader,
            net = net,
            epochs = model_cfg['epochs'],
            lr = model_cfg['lr'],
            momentum = model_cfg['momentum'],
            modelDir = model_cfg['modelDir'],
            device = device
        )
        
    elif 'test' in targets:
        with open('config/data-params.json') as fh:
            data_cfg = json.load(fh)
        # load the modelDir
        with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)
        
        # build paths
        TRAIN_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeTrain']
        VAL_PATH = data_cfg['dataDir'] + '/' + data_cfg['dataTypeVal']
        JSON_TRAIN_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeTrain'])
        JSON_VAL_PATH = data_cfg['annFile'].format(data_cfg['dataDir'] + '/anno2017', data_cfg['dataTypeVal'])
        
        # transforms 
        transform = transforms.Compose([ 
                transforms.Resize([240,240]),
                transforms.ToTensor()
        ])
        
        # load test data
        coco_test =  coco_dataloader.CocoDetection(root = VAL_PATH, annFile = JSON_VAL_PATH, transform = transform )
        testloader = torch.utils.data.DataLoader(coco_test, batch_size=5,
                               shuffle=True, num_workers=2)
        
        # change to gpu 
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print("COMPUTING AS: {}".format(device))
        
        net = model.Net()
        net = net.to(device)
        
        print('Loading model from {}...'.format(model_cfg['modelDir']))
        net.load_state_dict(torch.load(model_cfg['modelDir']))
        
        model.test_model(net, testloader, device)
        
        elif 'method7' in targets:
            with open('test/test-data-params.json') as fh:
                method7_config = json.load(fh)
                
            with open('config/model-params.json') as fh:
                model_cfg = json.load(fh)
                
            #load in data
            inputs, labels = test_model.load_testdata(method7_config['train_inputs'],method7_config['train_labels'])
            #transforms
            transform = transforms.Compose([transfroms.Resize([240, 240]), transforms.ToTensor()])
            
            # change to gpu
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            print('Computing as: {}'.format(device))
            
            net = model.Net()
            net = net.to(device)

            #run the model
            test_model.run_model(
                input_tensors = inputs,
                label_tensors = labels,
                net = net,
                epochs = model_cfg['epochs'],
                lr = model_cfg['lr'],
                momentum = model_cfg['momentum'],
                modelDir = m7['modelDir'],
                device = device)
if __name__ == '__main__':
    targets = sys.argv[1:]
    main(targets)